# Note Book For Seniment Analysis - EHmin

# Build DataFrame

 - for now, I use only 10000 data set

In [1]:
import pandas as pd 
df = pd.read_csv('review_lp31.csv')

In [2]:
df.head(5)

,stars,text
0,5,impressive decor start from the entrance outsi...
1,5,"after i work a crazy long day , hub be on dinn..."
2,3,breakfast bagel : 2.5 out of 5 pre make egg th...
3,5,fantastic burrito and taco ! this be a real ta...
4,5,country acre be amazing ! ! of all the petcos ...


In [3]:
df = df.rename(columns={'stars':'label','text':'text' })

In [4]:
small_df = df[0:1000]
small_df

,label,text
0,5,impressive decor start from the entrance outsi...
1,5,"after i work a crazy long day , hub be on dinn..."
2,3,breakfast bagel : 2.5 out of 5 pre make egg th...
3,5,fantastic burrito and taco ! this be a real ta...
4,5,country acre be amazing ! ! of all the petcos ...
...,...,...
995,5,i have the pleasure of work with eva today . s...
996,4,you ca n't go wrong with the french toast or l...
997,1,so back in march i go to auto air by ben to ge...
998,5,best waffle ever ! i build my own with nutella...


In [5]:
small_df['label'] = small_df['label'] - 1
small_df

/var/tmp/ipykernel_10586/2213859284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['label'] = small_df['label'] - 1


,label,text
0,4,impressive decor start from the entrance outsi...
1,4,"after i work a crazy long day , hub be on dinn..."
2,2,breakfast bagel : 2.5 out of 5 pre make egg th...
3,4,fantastic burrito and taco ! this be a real ta...
4,4,country acre be amazing ! ! of all the petcos ...
...,...,...
995,4,i have the pleasure of work with eva today . s...
996,3,you ca n't go wrong with the french toast or l...
997,0,so back in march i go to auto air by ben to ge...
998,4,best waffle ever ! i build my own with nutella...


In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(small_df , test_size=0.2, random_state=42)

In [7]:
from datasets import Dataset

small_train_dataset = Dataset.from_pandas(train_df)
small_eval_dataset = Dataset.from_pandas(test_df)
small_train_dataset[100]

/opt/conda/envs/EHmin/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'label': 3,
 'text': "bushi be one of my midtown favorite , and i can never understand why it 's always so quiet . the booth be typically fill with romantic couple while the center of the restaurant be ghostly quiet . when our cherry blossom roll be present , i have to stifle a squeal of delight . artfully display , it look like a colorful caterpillar with radish-sprout antenna , orange roe sprinkle , and avocado stripe . our nigiri ( crab and yellowtail ) be melt-in-your-mouth fresh , and even the cuke salad here be a delicious sesame-sweet piece of art pile tastefully and top with succulent string of crab and sprout . the yummy-yummy and playboy be outstanding here as well .",
 '__index_level_0__': 541}

# Tokenize

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

# Train

In [9]:
# !pip install transformers

In [10]:
#!pip install --user --upgrade tensorflow

In [11]:
#!pip install --user --upgrade protobuf

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [13]:
#!pip install protobuf==3.20

In [14]:
#pip install tensorboardX

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

# Evaluate

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
my = []

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch",
                                  logging_strategy = 'epoch',
                                  save_strategy = 'epoch',
                                  gradient_accumulation_steps=8,
                                  #logging_steps = 100,
                                  num_train_epochs=10
                                  )


In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch",
                                  gradient_accumulation_steps=8,
                                  eval_accumulation_steps = 8
                                  
                                  logging_strategy = 'epoch',
                                #   logging_strategy = 'steps',
                                #   logging_steps = 100,
                                  
                                  save_strategy = 'epoch',
                                #   save_strategy = 'steps',
                                #   save_steps = 1,
                                  
                                  num_train_epochs=10
                                  )

# Trainer

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11048989941011771972
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14433583104
locality {
  bus_id: 1
  links {
  }
}
incarnation: 60322462840827544
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
xla_global_id: 416903419
]


2023-06-09 03:21:25.651783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 03:21:25.653583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:21:25.654942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:21:25.655190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [22]:
from transformers import TrainerCallback
from copy import deepcopy

class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy
        
        
# trainer.add_callback(CustomCallback(trainer)) 

In [23]:
with tf.device("/device:GPU:0"):
     trainer.train()

2023-06-09 03:21:26.234160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:21:26.234533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:21:26.234725: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:21:26.235343: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 03:21:26.235567: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Epoch,Training Loss,Validation Loss


RuntimeError: [enforce fail at inline_container.cc:319] . unexpected pos 33152 vs 33104

In [ ]:
trainer.save_model('./')

In [ ]:
metrics = trainer.evaluate(small_eval_dataset)

In [ ]:
print(metrics)